This is modified from https://colab.research.google.com/drive/1PEQyJO1-f6j0S_XJ8DV50NkpzasXkrzd?usp=sharing#scrollTo=ib_We3NLtj2E

In [2]:
!nvidia-smi

Wed Apr  3 23:11:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:AF:00.0 Off |                    0 |
| N/A   38C    P0              41W / 300W |      3MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
import random

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    AutoConfig
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
config = AutoConfig.from_pretrained("/home/sds262_kg797/palmer_scratch/Llama-2-7b-chat-hf/config.json") # Replace with your own path
access_token = "xxx" # Replace with your own token

# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "b-mc2/sql-create-context"

# Fine-tuned model name
new_model = "llama-2-7b-sql"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=access_token)

In [3]:
dataset = load_dataset(dataset_name, split="train[0:10]")
dataset[0]

{'question': 'How many heads of the departments are older than 56 ?',
 'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'context': 'CREATE TABLE head (age INTEGER)'}

The prompt template for Llama 2:
```python
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

In [7]:
def format_sql(sample):
    context = f"<s>[INST] Here's the dataset overview: {sample['context']}." if len(sample['context']) > 0 else None
    instruction = f" Provide SQL that answer: {sample['question']}"
    reponse = f" [/INST] {sample['answer']}"
    # join all parts together
    prompt = "".join([i for i in [context, instruction, reponse] if i is not None])
    return prompt

def template_dataset(sample):
    sample['text'] = f"{format_sql(sample)}{tokenizer.eos_token}"
    return sample

In [13]:
import random

# Define multiple templates for formatting the sample
def template_a(sample):
    prefix = "<s>[INST]"
    if sample['context']:
        instruction = f"With the given context '{sample['context']}', how would you write SQL to retrieve data: {sample['question']}"
    else:
        instruction = f"How would you write SQL to answer: {sample['question']}"
    sql_markdown = f"```sql\n{sample['answer']}\n```"
    return f"{prefix} {instruction} [/INST] \n{sql_markdown}</s>"

def template_b(sample):
    prefix = "<s>[INST]"
    if sample['context']:
        instruction = f"How can the question '{sample['question']}' be answered using SQL, considering the context: '{sample['context']}'"
    else:
        instruction = f"How can the question '{sample['question']}' be answered using SQL"
    sql_markdown = f"```sql\n{sample['answer']}\n```"
    return f"{prefix} {instruction} [/INST] \n{sql_markdown}"

def template_c(sample):
    prefix = "<s>[INST]"
    if sample['context']:
        instruction = f"Dataset overview: '{sample['context']}'. Please demonstrate how SQL can be used to address the question: '{sample['question']}'"
    else:
        instruction = f"Please demonstrate how SQL can be used to address the question: '{sample['question']}'."
    sql_markdown = f"```sql\n{sample['answer']}\n```"
    return f"{prefix} {instruction} [/INST] \n{sql_markdown}"

def template_d(sample):
    prefix = "<s>[INST]"
    if sample['context']:
        instruction = f"Let's collaborate on writing an SQL query to solve for '{sample['question']}' within this context: '{sample['context']}'. How should we approach it"
    else:
        instruction = f"Let's collaborate on writing an SQL query to solve for '{sample['question']}'. How should we approach it?"
    sql_markdown = f"```sql\n{sample['answer']}\n```"
    return f"{prefix} {instruction} [/INST] \n{sql_markdown}"

def template_e(sample):
    prefix = "<s>[INST]"
    if sample['context']:
        instruction = f"As part of our data analysis with the background '{sample['context']}', could you formulate an SQL query to address the inquiry: '{sample['question']}'"
    else:
        instruction = f"Could you formulate an SQL query to address the inquiry: '{sample['question']}'"
    sql_markdown = f"```sql\n{sample['answer']}\n```"
    return f"{prefix} {instruction} [/INST] \n{sql_markdown}"

def template_e(sample):
    prefix = "<s>[INST]"
    if sample['context']:
        instruction = f"You've been tasked with finding information related to :'{sample['question']}', under this context: '{sample['context']}'"
    else:
        instruction = f"You've been tasked with finding information related to :'{sample['question']}'"
    sql_markdown = f"```sql\n{sample['answer']}\n```"
    return f"{prefix} {instruction} [/INST] \n{sql_markdown}"


# Function to randomly select a template and format the sample
def format_sample_randomly(sample):
    templates = [template_a, template_b, template_c, template_d, template_e]
    chosen_template = random.choice(templates)
    return chosen_template(sample)

# Modified template_dataset function that uses the new random format function
def template_dataset(sample, tokenizer):
    sample['text'] = f"{format_sample_randomly(sample)}\n{tokenizer.eos_token}"
    return sample

In [14]:
dataset_llama = dataset.map(template_dataset, fn_kwargs={'tokenizer': tokenizer}, remove_columns=list(dataset.features))
dataset_llama[0]['text']
print(dataset_llama[0]['text'])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

<s>[INST] You've been tasked with finding information related to :'How many heads of the departments are older than 56 ?', under this context: 'CREATE TABLE head (age INTEGER)' [/INST] 
```sql
SELECT COUNT(*) FROM head WHERE age > 56
```
</s>


In [9]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_llama,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sds262_kg797/.conda/envs/Llama/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/home/sds262_kg797/.conda/envs/Llama/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss


/home/sds262_kg797/.conda/envs/Llama/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-660e1bf7-5195b31d394ba4f41543c0b7;5d712a06-3b50-4646-a63c-11131b9b2bc5)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Repo model meta-llama/Llama-2-7b-chat-hf is gated. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-chat-hf.
  warnings.warn(
/home/sds262_kg797/.conda/envs/Llama/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
!conda install tensorboardX

In [10]:
!ls

LICENSE        chatbot.py	finetune.ipynb	    llama-2-7b-sql
README.assets  chatbot_app.py	inference.ipynb     models
README.md      config.yaml	inference_local.py  rag_chatbot.py
__pycache__    environment.yml	langchain.ipynb     results


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
prompt = """ [INST] Create table If Not Exists Employee (id int, name varchar(255), salary int, departmentId int) Create table If Not Exists Department (id int, name varchar(255)) A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.
Write a solution to find the employees who are high earners in each of the departments.
Return the result table in any order.
The result format is in the following example. [/INST]
"""

model_inputs = tokenizer(prompt, return_tensors="pt").to(device)
output = model.generate(**model_inputs)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Using device: cuda
 [INST] Create table If Not Exists Employee (id int, name varchar(255), salary int, departmentId int) Create table If Not Exists Department (id int, name varchar(255)) A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.
Write a solution to find the employees who are high earners in each of the departments.
Return the result table in any order.
The result format is in the following example. [/INST]
To find the high earners in each department, we need to first create the two tables `Employee` and `Department`, and then write a query to join these tables and identify the employees who are high earners in each department.

Here is the SQL code to create the `Employee` and `Department` tables:
```sql
CREATE TABLE Employee (
  id INT,
  name VARCHAR(255),
  salary INT
);

CREATE TABLE Department (
  i